In [1]:
# This notebook just creates data for use in ./MergeVsJoinDataPrep.ipynb.

In [26]:
import os
import uuid
import pandas as pd
import math
import random

In [27]:
 # day 1
def product_names(i):
    if i % 3 == 0:
        return f'widget{i}'
    elif i % 3 == 1:
        return f'sproket{i}'
    else:
        return f'doodad{i}'

def build_products(n, j=0):
    return [
    {'product_id': f'p_{i+j}', 'product_name': product_names(i+j), 'price': round(random.random() * 100, 2)}
    for i in range(n)
]
    
products = build_products(125, 0)

def build_invoices(n, j):
    return [
        {'invoice_id': f'inv_{i+j}', 'customer': str(uuid.uuid4()), 'status': 'draft' if random.random() < 0.1 else 'sent'}
        for i in range(n)
    ]

def build_invoice_items(invoices, products):
    local_items = []
    for invoice in invoices:
        num_items = random.randint(1, 5)
        items = [
            {'invoice_item_id': str(uuid.uuid4()), 
             'invoice': invoice['invoice_id'], 
             'count': random.randint(1, 4), 
             'product': random.choice(products)['product_id']}
            for i in range(num_items)
        ]
        local_items.extend(items)
    return local_items
    
invoices = build_invoices(1000, 0)
invoice_items = build_invoice_items(invoices, products)
    

In [28]:
# day 1 - any invoice that has changed will be in here.
# separate delete records made.
existing_invoices = {k['invoice_id']: k for k in invoices}
existing_items = {}
for item in invoice_items:
    inv_id = item['invoice']
    if inv_id not in existing_items:
        existing_items[inv_id] = []
    existing_items[inv_id].append(item)


# update products
updated_products = random.sample(products, 25)
for p in updated_products:
    p['price'] = round(random.random() * 100, 2)
updated_products.extend(build_products(25, 200))

# make new invoices
random_invoice_ids = set([random.randint(0, 5000) for r in range(500)])  # gives us updates to 20%
changed_invoices = [
    {'invoice_id': f'inv_{i}', 
     'customer': str(uuid.uuid4()), 
     'status': 'draft' if random.random() < 0.1 else 'sent'}
    for i in random_invoice_ids if i > 1000
]

# make some new invoices.
new_invoices = build_invoices(500, 1001)
changed_invoice_items = build_invoice_items(new_invoices, updated_products)

deleted_invoice_items = []
deleted_invoices = []

# update some invoices [drop item, add item, modify item]
invoices_to_update = [i for i in random_invoice_ids if i <= 1000]
for update_i in invoices_to_update:
    rand_action = random.random()
    key = f'inv_{update_i}'
    if rand_action < .05:
        # drop an invoice
        deleted_invoices.append(existing_invoices[key])
    elif rand_action < 0.1:
        # drop an invoice item or two
        num_to_drop = 1 if random.random() < 0.6 else 2
        to_drop = random.choice(existing_items[key])
    elif rand_action < 0.9:
        inv = existing_invoices[key]
        inv['status'] == 'sent'
        changed_invoices.append(inv)
    else:
        # find an item and update the count
        inv_item = random.choice(existing_items[key])
        inv_item['count'] = random.randint(1, 6)
        changed_invoice_items.append(inv_item)


In [29]:
os.makedirs('./data/products/updates/day=0', exist_ok=True)
pd.DataFrame(products).to_csv("./data/products/updates/day=0/products.csv")

os.makedirs('./data/invoice/updates/day=0', exist_ok=True)
pd.DataFrame(invoices).to_csv("./data/invoice/create/day=0/invoices.csv")

os.makedirs('./data/invoiceitems/updates/day=0', exist_ok=True)
pd.DataFrame(invoice_items).to_csv("./data/invoiceitems/updates/day=0/invoiceitems.csv")



OSError: Cannot save file into a non-existent directory: 'data\invoice\create\day=0'

In [25]:
os.makedirs('./data/products/updates/day=1', exist_ok=True)
pd.DataFrame(updated_products).to_csv("./data/products/updates/day=1/products.csv")

os.makedirs('./data/invoice/updates/day=1', exist_ok=True)
pd.DataFrame(changed_invoices).to_csv("./data/invoice/updates/day=1/invoices.csv")
os.makedirs('./data/invoice/deletes/day=1', exist_ok=True)
pd.DataFrame(deleted_invoices).to_csv("./data/invoice/deletes/day=1/invoices.csv")

os.makedirs('./data/invoiceitems/updates/day=1', exist_ok=True)
os.makedirs('./data/invoiceitems/deletes/day=1', exist_ok=True)
pd.DataFrame(changed_invoice_items).to_csv("./data/invoiceitems/updates/day=1/invoiceitems.csv")
pd.DataFrame(deleted_invoice_items).to_csv("./data/invoiceitems/deletes/day=1/invoiceitems.csv")



[{'invoice_item_id': 'f2433673-41ac-44f7-a31c-a321849f1312',
  'invoice': 'inv_1001',
  'count': 4,
  'product': 'p_88'},
 {'invoice_item_id': 'd2fa21c8-9422-4c00-94ce-c1521405c1de',
  'invoice': 'inv_1001',
  'count': 1,
  'product': 'p_218'},
 {'invoice_item_id': '80449028-7bf3-466d-9afd-fc7ff6c598c0',
  'invoice': 'inv_1001',
  'count': 3,
  'product': 'p_200'},
 {'invoice_item_id': 'c1ee38fa-9bd9-45a6-9592-97ab63f57c3f',
  'invoice': 'inv_1001',
  'count': 3,
  'product': 'p_67'},
 {'invoice_item_id': '29915e37-89bd-44fd-a09b-01e436dc03e0',
  'invoice': 'inv_1001',
  'count': 2,
  'product': 'p_211'},
 {'invoice_item_id': '7d77f80c-1c94-49dc-ab02-9974473e8889',
  'invoice': 'inv_1002',
  'count': 1,
  'product': 'p_74'},
 {'invoice_item_id': '55430b55-710f-4a2a-a4d1-5d18cd7bfe1c',
  'invoice': 'inv_1002',
  'count': 3,
  'product': 'p_49'},
 {'invoice_item_id': '2bf47cc7-0a7f-4cdc-9128-59ae468f3c5f',
  'invoice': 'inv_1002',
  'count': 4,
  'product': 'p_66'},
 {'invoice_item_id': 

In [11]:
random.sample([1, 2, 3], 2)

[1, 3]